In [1]:
import pdfplumber
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import re
import string
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import bookChapterPageNumbers as chaps

In [2]:

def cleanText(page_text):
    
    clean_text = page_text
    
    # remove punctuation
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', clean_text)
    # make lower case
    clean_text = clean_text.lower()
    
    # remove numbers
    clean_text = re.sub('\w*\d\w*', ' ', clean_text)
      
    # tokenize the text either in individual words or sentences
    clean_text = word_tokenize(clean_text)
#     page_sentences = sent_tokenize(clean_text) # this doesn't work if punctuation has already been removed, can't clean the punctuation with the same line of code as above once it's been tokenized

    # remove punctuation which has been reintroduced after tokenizing
    clean_text = [word for word in clean_text if word.isalpha()]

    # remove stop words (can do this manually or automatically with the vectorizer)
    clean_text = [word for word in clean_text if word not in stop_words]
    
    # stem words
    clean_text = [porter.stem(word) for word in clean_text]

    
#     print(clean_text)
#     print(page_sentences)
    
#     print('Num words on page = ', len(page_words))
#     print('Num sentences on page = ', len(page_sentences))
    
#     twograms = list(ngrams(page_words,2))
#     print(twograms)

    return clean_text
    

    

In [3]:
# books = ["../StormlightArchiveBooks/The_Way_of_Kings_The_Stormlight_Archive_Book_1.pdf"]
books = ["../StormlightArchiveBooks/The_Way_of_Kings_The_Stormlight_Archive_Book_1.pdf", "../StormlightArchiveBooks/Words_of_Radiance_The_Stormlight_Archive_Book_2.pdf"]

book_chapters_pages = [chaps.book_1_chapter_pages, chaps.book_2_chapter_pages, chaps.book_3_chapter_pages, chaps.book_4_chapter_pages]

In [4]:
list_of_book_dfs = []

for i, book in enumerate(books):
    with pdfplumber.open(book) as pdf:
        this_book_chaps_pages = book_chapters_pages[i]
        
        df_text_book = pd.DataFrame()
        book_chapters_text = []
        
        num_distinct_chapters = len(this_book_chaps_pages)-1
        
        for chap in range(0,num_distinct_chapters):
            startingPage = this_book_chaps_pages[chap]-1 # minus one to shift pages down by 1 since the pdf.pages object starts at 0
            endingPage = this_book_chaps_pages[chap+1]-1-1 # extra minus one for ending page to not include beginning of next chapter
            print('book: ', i+1, ' chap: ', chap+1, ' starting page:', startingPage, ' ending page: ', endingPage)

            chapter_text = ''
            
            for pageNum in range(startingPage, endingPage+1):
#                 print('chap ', chap, ' page ', pageNum)
                
                this_page = pdf.pages[pageNum]
                page_text = this_page.extract_text()
                
                if page_text is None: continue
                
                cleaned_page_text = cleanText(page_text)
                recombined_text = ' '.join(cleaned_page_text)
                
                chapter_text += ' '
                chapter_text += recombined_text
                
            book_chapters_text.append(chapter_text)
    
        df_text_book['chapter_text'] = book_chapters_text
        df_text_book['book'] = [i+1] * num_distinct_chapters
        df_text_book['chapter'] = [n+1 for n in range(0, num_distinct_chapters)]
        
#         print(df_text_book)
        list_of_book_dfs.append(df_text_book)
        
        
df_text_all_books = pd.concat(list_of_book_dfs, ignore_index=True)
# print(df_text_all_books)

df_text_all_books.to_pickle("./sla_chapter_text.pkl")
    
# print(len(all_books_chapters_text))
        
#         this_page = pdf.pages[17]
#         page_text = this_page.extract_text()
#         print(page_text)

#         cleaned_page_text = cleanText(page_text)
#         print(cleaned_page_text)

#         recombined_text = ' '.join(cleaned_page_text)
#         print(recombined_text)

    #     vectorizeText(recombined_text)
    

book:  1  chap:  1  starting page: 11  ending page:  16
book:  1  chap:  2  starting page: 17  ending page:  35
                                        chapter_text  book  chapter
0   prelud stormlight archiv kalak round rocki st...     1        1
1   love men frigid thing mountain stream three s...     1        2
book:  2  chap:  1  starting page: 20  ending page:  39
book:  2  chap:  2  starting page: 40  ending page:  65
                                        chapter_text  book  chapter
0   six year ago jasnah kholin pretend enjoy part...     2        1
1   perfectli frank happen last two month upon he...     2        2
                                        chapter_text  book  chapter
0   prelud stormlight archiv kalak round rocki st...     1        1
1   love men frigid thing mountain stream three s...     1        2
2   six year ago jasnah kholin pretend enjoy part...     2        1
3   perfectli frank happen last two month upon he...     2        2
